In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

import PortOpt.invariants as invs
import PortOpt.moment_est as moments
from PortOpt.opt_allocations import Optimiser

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load in Data

In [27]:
def get_data(tickers, interval, start, end):
    full_prices = pd.DataFrame()
    for i in tickers:
        df = yf.download(i, interval=interval, start=start, end=end)['Adj Close']
        print(f'{i} data downloaded...')

        full_prices = pd.concat([full_prices, df], axis=1, sort=False)

    full_prices.columns = tickers
      
    return full_prices

In [28]:
tickers = ['AAPL', 'MSFT', 'CVX', 'GE', 'GOOGL']

stock_data = get_data(tickers, '1d', '2015-01-01', '2021-01-01')

[*********************100%***********************]  1 of 1 completed
AAPL data downloaded...
[*********************100%***********************]  1 of 1 completed
MSFT data downloaded...
[*********************100%***********************]  1 of 1 completed
CVX data downloaded...
[*********************100%***********************]  1 of 1 completed
GE data downloaded...
[*********************100%***********************]  1 of 1 completed
GOOGL data downloaded...


### Get Expected Returns

In [29]:
stock_returns = invs.stock_invariants(stock_data)
stock_returns.head()

,AAPL,MSFT,CVX,GE,GOOGL
2014-12-31 00:00:00,-0.009558,0.006651,0.003559,-0.008345,-0.002094
2015-01-02 00:00:00,-0.028576,-0.009238,-0.040792,-0.018527,-0.019238
2015-01-05 00:00:00,0.000094,-0.014786,-0.000463,-0.021780,-0.024989
2015-01-06 00:00:00,0.013925,0.012625,-0.000833,0.000415,-0.002945
2015-01-07 00:00:00,0.037702,0.028994,0.022625,0.011971,0.003478


In [59]:
exp_returns = stock_returns.mean() * 252
exp_returns

AAPL     0.277458
MSFT     0.280950
CVX     -0.004048
GE      -0.112725
GOOGL    0.199258
dtype: float64

### Get Covariance Matrix

In [31]:
stock_cov = moments.exp_cov(stock_returns)
stock_cov

,AAPL,MSFT,CVX,GE,GOOGL
AAPL,0.153987,0.097490,0.048077,0.030008,0.075093
MSFT,0.097490,0.109694,0.051199,0.033677,0.081058
CVX,0.048077,0.051199,0.223468,0.152176,0.063097
GE,0.030008,0.033677,0.152176,0.252910,0.045842
GOOGL,0.075093,0.081058,0.063097,0.045842,0.095350


### Solve Optimisation Problem

In [113]:
optimiser = Optimiser(tickers, exp_returns, stock_cov)
# optimiser.mean_variance(threshold=0.1)
weights = optimiser.max_sharpe()

# min_pos = lambda x: x >= 0.1*np.ones(len(tickers))
# optimiser.add_constraint(min_pos)

# weights = optimiser.solve(objective='max_sharpe')
weights

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Sep 29 12:26:52 AM: Your problem has 6 variables, 4 constraints, and 0 parameters.
(CVXPY) Sep 29 12:26:52 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 29 12:26:52 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 29 12:26:52 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 29 12:26:52 AM: Compiling problem (target solver=OSQP).
(CVXPY) Sep 29 12:26:52 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing 

AAPL     1.566419e-01
MSFT     8.433581e-01
CVX     -5.993091e-24
GE       4.644015e-23
GOOGL   -3.322861e-23
dtype: float64